In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
train_set = pd.read_hdf("train_set0517.h5")
test_set = pd.read_hdf("test_set0517.h5")

In [3]:
x_train_data = train_set[['YEAR', 'MONTH', 'DAY', 'TMAX', 'TMIN', 'TAVG', 'PRCP']]
x_train_data = x_train_data.drop(x_train_data.index[-1])
x_test_data = test_set[['YEAR', 'MONTH', 'DAY', 'TMAX', 'TMIN', 'TAVG', 'PRCP']]
x_test_data = x_test_data.drop(x_test_data.index[-1])

In [4]:
from tensorflow.keras import backend as K

D:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; 

In [5]:
x_train = K.cast_to_floatx(x_train_data.values.reshape((x_train_data.shape[0], 1, x_train_data.shape[1])))
x_test = K.cast_to_floatx(x_test_data.values.reshape((x_test_data.shape[0], 1, x_test_data.shape[1])))

In [6]:
y_train_PRCP_data = train_set[['PRCP_Interval']].astype(int)
y_train_PRCP = y_train_PRCP_data.drop(y_train_PRCP_data.index[0])

y_test_PRCP_data = test_set[['PRCP_Interval']].astype(int)
y_test_PRCP = y_test_PRCP_data.drop(y_test_PRCP_data.index[0])

In [7]:
y_train_PRCP[y_train_PRCP['PRCP_Interval'] < 0] = 0

In [37]:
y_train_PRCP[y_train_PRCP['PRCP_Interval'] > 500]

,PRCP_Interval
86290,926
86486,725
94415,872
96149,744
96533,741
98404,941
121131,878
121149,753
179256,522
236275,501


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU

In [9]:
PRCP_model = Sequential()
PRCP_model.add(GRU(1400, dropout = 0.25, recurrent_dropout = 0.25, input_shape = (
    x_train.shape[1], x_train.shape[2])))
PRCP_model.add(Dense(1400, activation = 'sigmoid'))
PRCP_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
PRCP_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 1400)              5913600   
_________________________________________________________________
dense (Dense)                (None, 1400)              1961400   
Total params: 7,875,000
Trainable params: 7,875,000
Non-trainable params: 0
_________________________________________________________________


In [64]:
history = PRCP_model.fit(x_train, y_train_PRCP, batch_size = 50, epochs = 10, validation_data = (x_test, y_test_PRCP))

Train on 4060570 samples, validate on 414830 samples
Epoch 1/10
4060570/4060570 [==============================] - 11456s 3ms/sample - loss: 1.2817 - acc: 0.7762 - val_loss: 1.3361 - val_acc: 0.7588
Epoch 2/10
4060570/4060570 [==============================] - 11433s 3ms/sample - loss: 1.2651 - acc: 0.7759 - val_loss: 1.3223 - val_acc: 0.7588
Epoch 3/10
4060570/4060570 [==============================] - 11458s 3ms/sample - loss: 1.2634 - acc: 0.7759 - val_loss: 1.3123 - val_acc: 0.7588
Epoch 4/10
 321600/4060570 [=>............................] - ETA: 2:52:22 - loss: 1.2701 - acc: 0.7750

KeyboardInterrupt: 

In [66]:
PRCP_model.save_weights('PRCP_model.h5')

In [10]:
PRCP_model.load_weights('PRCP_model.h5')

In [11]:
predict_data = pd.read_csv('氣象局觀測資料.csv')
predict_data = predict_data[['YEAR', 'MONTH', 'DAY', 'TMAX', 'TMIN', 'TAVG', 'PRCP']]
predict_data = K.cast_to_floatx(predict_data.values.reshape(predict_data.shape[0], 1, predict_data.shape[1]))

In [12]:
PRCP_result = PRCP_model.predict_classes(predict_data)
PRCP_result

array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)